<a href="https://colab.research.google.com/github/FawneeCakes/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,889 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
display(application_df.head())

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for "other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for "other"
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_type = class_value_counts.loc[class_value_counts > 1]
class_type

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_value_counts [class_value_counts <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
display(dummies_df.head())

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [11]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y,test_size=.2)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Optimization 1 - Adding More Neurons**

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 80
input_features = len(X_train_scaled[0])
nn_model1 = tf.keras.models.Sequential()

# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Output layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10081 (39.38 KB)
Trainable params: 10081 (39.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
858/858 [==============================] - 3s 2ms/step - loss: 0.5709 - accuracy: 0.7216
Epoch 2/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7263
Epoch 3/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7285
Epoch 4/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 5/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7299
Epoch 6/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7310
Epoch 7/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7304
Epoch 8/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7320
Epoch 9/200
858/858 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7302
Epoch 10/200
858/858 [==============================] - 2s 2ms/step - loss: 0.5474 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 2s - loss: 0.5708 - accuracy: 0.7389 - 2s/epoch - 8ms/step
Loss: 0.5707989931106567, Accuracy: 0.7389212846755981


**Optimization 2 - Adding More Layers**

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 30
units_3 = 20
input_features = len(X_train_scaled[0])
nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_3, activation = "relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 20)                620       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 6591 (25.75 KB)
Trainable params: 6591 (25.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5706 - accuracy: 0.7192
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7277
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7276
Epoch 4/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7285
Epoch 5/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5500 - accuracy: 0.7297
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7308
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7313
Epoch 8/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7322
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7317
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5468 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5541 - accuracy: 0.7366 - 761ms/epoch - 4ms/step
Loss: 0.5540915727615356, Accuracy: 0.7365888953208923


**Optimization 3 - Using Alertnate Activation Functions - tanh**

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 30
input_features = len(X_train_scaled[0])
nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_2, activation = "tanh"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5677 - accuracy: 0.7220
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7273
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7283
Epoch 4/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7275
Epoch 5/100
858/858 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7286
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7297
Epoch 7/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7293
Epoch 8/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7309
Epoch 9/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7295
Epoch 10/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5479 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5494 - accuracy: 0.7372 - 738ms/epoch - 3ms/step
Loss: 0.5494145750999451, Accuracy: 0.73717200756073


In [32]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model4 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model4.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model4.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model4.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model4

**Optimization 4 - Automated Optimizer**

In [34]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [36]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [37]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.7380466461181641

Best val_accuracy So Far: 0.7399417161941528
Total elapsed time: 00h 26m 28s


In [38]:
# Export our model to HDF5 file
from google.colab import files

nn_model3.save('/content/AlphabetSoupCharity_Optimization.h5') # Use nn_model3 instead of nn
files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>